# SHAKEtime: Time-Dependent SHAKEmap Evolution and Comparison

This notebook demonstrates how to use the `SHAKEtime` class to analyze the
**temporal evolution of USGS SHAKEmap products** for a single earthquake event.

It is designed as a **core usage and analysis notebook**, focusing on how
shaking estimates change across successive SHAKEmap versions as new
information becomes available.

---

## Overview

Earthquake impact assessments evolve rapidly following an event as additional
data are incorporated, including new seismic stations, macroseismic reports,
and updated source models. SHAKEmap products reflect this process through a
sequence of published versions.

The `SHAKEtime` class provides a unified framework to ingest, align, and compare
these versions in space and time, enabling systematic analysis of shaking
evolution and consistency.

This notebook walks through the **standard SHAKEtime workflow**, from data
ingestion to spatial comparison and temporal diagnostics.

---

## Core Functionality

- Versioned ingestion of SHAKEmap products  
- Unified spatial grid construction for cross-version comparison  
- Spatial delta mapping between SHAKEmap versions  
- Temporal evolution analysis of shaking metrics  
- Diagnostic and visualization tools  
- Optional PAGER parsing and auxiliary analyses  

---

## Dependencies

This notebook assumes the following Python environment:

`Python 3.x` `numpy` `pandas` `scipy` `matplotlib` `xml (built-in)`

Optional (depending on plots and extensions):
`cartopy` `geopandas`

Custom project modules:
`SHAKEtime` `SHAKEparser` `SHAKEmapper` `SHAKEtools`

---

`Last update: January, 2026` `SHAKEtime version: 26.1.4`


# Event Overview 

## Notebook Setup  
### Dock files Shakemap 

In [ ]:
base_folder = 'event_data/SHAKEfetch'

event_id = 'us6000jlqa'         #'us7000pn9s' Myanmar ; 'us7000m9g4' Taiwan ; 'us6000jllz' Turkey 7.8 ;'us6000jlqa' Turkey 7.5 

if event_id == 'us7000pn9s': #Myanmar #
    event_id = 'us7000pn9s'
    event_epicenter_lon = 95.925
    event_epicenter_lat = 22.001
    version = '022'
    first_version = '001'
    event_time = '2025-03-28 06:20:52'

    shakemap_folder = "./event_data/SHAKEfetch/usgs-shakemap-versions"
    pager_folder = "./event_data/SHAKEfetch/usgs-pager-versions"
    rupture_folder = "./event_data/SHAKEfetch/usgs-rupture-versions"
    stations_folder = "./event_data/SHAKEfetch/usgs-instruments_data-versions"
    version_list = ['001', '002','003', '004','005','006', '007', '008','009', '011', '012','013','014','015','016','017','018','019','020','021','022']
    selected_cities = ['Mandalay', 'Nay Pyi Taw','Chiang Mai' ,'Taungoo','Bangkok']

    xml_shakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{version}_grid.xml"
    xml_firstshakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{first_version}_grid.xml"

    file_path_rupturedata = f'./{base_folder}/usgs-rupture-versions/{event_id}/{event_id}_us_{version}_rupture.json'  # Replace with actual file path
    
    file_path_instrument_data = f"./{base_folder}/usgs-instruments_data-versions/{event_id}/{event_id}_us_{version}_stationlist.json"
    file_path_dyfixml = f"./{base_folder}/usgs-dyfi-versions/{event_id}/us7000pn9s_us_1_cdi_geo_1km.txt"
    
    all_dists = (
            'normal',
            'lognorm',
            'gamma',
            'weibull',
            'burr')

    
elif event_id == 'us6000jllz': #Turkey 7.8
    event_id = 'us6000jllz'
    event_epicenter_lon = 37.014
    event_epicenter_lat = 37.226
    version = '017'
    first_version = '001'

    event_time = '2023-02-06 01:17:34'
    
    shakemap_folder = "./event_data/SHAKEfetch/usgs-shakemap-versions"
    pager_folder = "./event_data/SHAKEfetch/usgs-pager-versions"
    rupture_folder = "./event_data/SHAKEfetch/usgs-rupture-versions"
    stations_folder = "./event_data/SHAKEfetch/usgs-instruments_data-versions"
    
    version_list = ['001', '002','003', '004','006', '007', '008','009', '011', '012']
    version_list = ['001', '002','003', '004','005','006', '007', '008','009', '010','011', '012','013','014','015','016','017']
    selected_cities = ['Elbistan','Gaziantep', 'Sanliurfa','Kahramanmaras' ,'Kilis','Narli','Malatya','Diyarbakir','Antakya' ,'Adana','Aleppo','Elazig','Iskenderun']
    xml_shakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{version}_grid.xml"
    xml_firstshakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{first_version}_grid.xml"

    file_path_rupturedata = f'./{base_folder}/usgs-rupture-versions/{event_id}/{event_id}_us_{version}_rupture.json'  # Replace with actual file path
    file_path_instrument_data = f"./{base_folder}/usgs-instruments_data-versions/{event_id}/{event_id}_us_{version}_stationlist.json"
    file_path_dyfixml = f"./{base_folder}/usgs-dyfi-versions/{event_id}/{event_id}_us_1_cdi_geo_1km.txt"

    all_dists = ( #Taiwan
        'normal',
        'lognorm',
        'gamma',
        'beta',
        'genextreme',
        'pearson3',
        'burr')

elif event_id == 'us6000jlqa': #Turkey 7.5
    event_id = 'us6000jlqa'
    event_epicenter_lon = 37.196
    event_epicenter_lat = 38.011
    version = '016'
    first_version = '001'

    event_time = '2023-02-06 10:24:48'
    
    shakemap_folder = "./event_data/SHAKEfetch/usgs-shakemap-versions"
    pager_folder = "./event_data/SHAKEfetch/usgs-pager-versions"
    rupture_folder = "./event_data/SHAKEfetch/usgs-rupture-versions"
    stations_folder = "./event_data/SHAKEfetch/usgs-instruments_data-versions"
    
    version_list = ['001', '002','003', '004','005','006', '007', '008','009', '010','011', '012','013','014','015','016']
    selected_cities = ['Gaziantep', 'Sanliurfa','Kahramanmaras' ,'Kilis','Narli','Malatya','Diyarbakir','Antakya' ,'Adana','Aleppo','Elazig','Iskenderun']
    xml_shakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{version}_grid.xml"
    xml_firstshakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{first_version}_grid.xml"

    file_path_rupturedata = f'./{base_folder}/usgs-rupture-versions/{event_id}/{event_id}_us_{version}_rupture.json'  # Replace with actual file path
    file_path_instrument_data = f"./{base_folder}/usgs-instruments_data-versions/{event_id}/{event_id}_us_{version}_stationlist.json"
    file_path_dyfixml = f"./{base_folder}/usgs-dyfi-versions/{event_id}/{event_id}_us_1_cdi_geo_1km.txt"

    all_dists = (
        'normal',
        'lognorm',
        'gamma',
        'beta',
        'genextreme',
        'pearson3',
        'burr')


elif event_id == 'us7000m9g4': #Taiwan

    event_id = 'us7000m9g4'
    version = '012'
    first_version = '001'
    event_epicenter_lon = 121.598
    event_epicenter_lat = 23.835


    
    xml_shakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{version}_grid.xml"
    xml_firstshakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{first_version}_grid.xml"

    file_path_rupturedata = f'./{base_folder}/usgs-rupture-versions/{event_id}/{event_id}_us_{version}_rupture.json'  # Replace with actual file path
    
    file_path_instrument_data = f"./{base_folder}/usgs-instruments_data-versions/{event_id}/{event_id}_us_{version}_stationlist.json"
    file_path_dyfixml = f"./{base_folder}/usgs-dyfi-versions/{event_id}/{event_id}_us_3_cdi_geo_1km.txt"

    event_time = '2024-04-02 23:58:12'
    
    shakemap_folder = "./event_data/SHAKEfetch/usgs-shakemap-versions"
    pager_folder = "./event_data/SHAKEfetch/usgs-pager-versions"
    rupture_folder = "./event_data/SHAKEfetch/usgs-rupture-versions"
    stations_folder = "./event_data/SHAKEfetch/usgs-instruments_data-versions"
    
    version_list = ['001', '002','003', '004','006', '007', '008','009', '011', '012','013','014']
    
    selected_cities = ['Hualien City', 'Yilan', 'Banqiao','Taipei' ,'Taoyuan','Keelung']

    all_dists = ( #Taiwan
            'normal',
            'lognorm',
            'gamma',
            'weibull',
            'expon',
            'beta',
            'genextreme',
            'pearson3',
            'burr')



### Behaviour Settings 


In [ ]:
#Event Overview 
instruments_data_map = False 
dyfi_stationslist_data_map = False
dyfi_CDI_data_map = False
plot_shakemap_v1 = False 
plot_shakemap_stationslist_vf = False 
plot_shakemap_CDI_vf = False
plot_pgashakemap_vf = False 


#settings 
summary_cache = True
unigrid_cache = True
aux_cache = True

unified_grid_test = False   		#used in thesis 

plot_ratemaps = True		 	#used 
plot_shakemaps = False			#used 
plot_shakemaps_std = False		 #used

plot_pager =False #				#used 
plot_hazard_thd = False#            #used 
plot_overview_panels = False #       # not used 
plot_auxiliary = False #
quantify_evolution = False #
create_evolution_panel = False          #used 

quantify_data_influence = False

compute_global_choas = False

uncertainty_quantification = True 
target_uq_decay = True

In [ ]:
from modules.SHAKEparser import *
from modules.SHAKEmapper import *
from modules.SHAKEtools import *



import os

output_path = f'./export/SHAKEtime/{event_id}/seismic_record_assessments'
os.makedirs(output_path, exist_ok=True)




In [ ]:
shakemap_parser = USGSParser(parser_type="shakemap_xml", imt='mmi', xml_file=xml_shakemap_file_path)
shakemap_metadata = shakemap_parser.get_metadata()

map_extent = [shakemap_metadata['grid_specification']['lon_min'], shakemap_metadata['grid_specification']['lon_max'],
              shakemap_metadata['grid_specification']['lat_min'], shakemap_metadata['grid_specification']['lat_max']]

map_extent = [float(s) for s in map_extent]


## Get Rupture Data 

In [ ]:
#Parse Rupture data 
rupture_parser = USGSParser(parser_type='rupture_json', mode='parse',rupture_json=file_path_rupturedata)
# Extract rupture coordinates from the parser.
rupture_x_coords, rupture_y_coords = rupture_parser.get_rupture_xy()

# Plotting function
fig, ax = plt.subplots()
ax.plot(rupture_x_coords, rupture_y_coords, '-', label='Rupture Extent')
ax.set_title('Rupture Dimensions Plot')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()
plt.show()

## Get Instruments Data 

In [ ]:
# Parse Instruments Data 
instrument_parser = USGSParser(parser_type = 'instrumented_data',json_file=file_path_instrument_data)
instruments_data = instrument_parser.get_dataframe(value_type='pga')

print(instruments_data.columns)

# Show maximum values 
# Step 1: Find the index of the maximum value in the 'pga' column
max_pga_index = instruments_data['pga'].idxmax()

# Step 2: Retrieve and print the row with the maximum 'pga'
max_pga_row = instruments_data.loc[max_pga_index]
print("Row with the maximum PGA value:")
print(max_pga_row)

instruments_data['predictions'][1]

In [ ]:
from modules.SHAKEtools import AccelerationUnitConverter

# convert units
converter = AccelerationUnitConverter()
instruments_data['HNE'] = converter.convert_unit(instruments_data['HNE'],'%g','cm/s2')
instruments_data['HNN'] = converter.convert_unit(instruments_data['HNN'],'%g','cm/s2')
instruments_data['HNZ'] = converter.convert_unit(instruments_data['HNZ'],'%g','cm/s2')
instruments_data['pga'] = converter.convert_unit(instruments_data['pga'],'%g','cm/s2')
instruments_data['pga_selected'] = converter.convert_unit(instruments_data['pga'].replace('null', np.nan),'%g','cm/s2')


instruments_data['pga_unit'] = 'cm/s2'

### Instruments Data Statistical Analysis 

In [ ]:
if instruments_data_map:
    import matplotlib.ticker as ticker

    marker_size = 10


    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    plt.plot(instruments_data['distance'],
                                  instruments_data['HNZ'],
                                  'k^',mfc='k',markersize=marker_size, label = 'Vertical component')
    
    
    # Compute the max of HNE and HNN for each row
    max_H = instruments_data[['HNE', 'HNN']].min(axis=1)  # :contentReference[oaicite:0]{index=0}
    plt.plot(instruments_data['distance'],
                                  max_H,
                                  'k^',mfc='yellow',markersize=marker_size, label = 'Horizontal component')
    
    plt.plot(instruments_data['distance'],
                                  instruments_data['pga_selected'].replace('null', np.nan),
                                  'k^',mfc='r',markersize=marker_size, label = 'PGA')
    
    
    plt.grid()
    plt.legend()
    
    # set axis scale 
    plt.xscale('log', base=10)
    plt.yscale('log',base=10)
    
    # Customize tick labels to scalar format
    ax = plt.gca()
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
    ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
    
    # Ensure that every tick label is formatted in scalar (no scientific notation)
    ax.xaxis.get_major_formatter().set_scientific(False)
    ax.yaxis.get_major_formatter().set_scientific(False)
    
    ax.set_axisbelow(True)
    ax.grid(
        True,
        linestyle='--',
        color='gray',
        linewidth=0.3,
        alpha=0.7
    )
    
    #set label sizes 
    plt.xlabel('Epicentral Distance [$km$]')
    plt.ylabel('PGA [$cm/s^2$]')
    
    #axis bounds 
    ticksx = [10,30, 40, 60, 100, 200, 300,500]
    ticksy = [10,100,500]
    
    plt.xticks(ticksx)
    plt.yticks(ticksy)
    
    
    plt.savefig(f"{output_path}/{event_id}_SeismicStationrecords_{version}.png", bbox_inches='tight')
    plt.savefig(f"{output_path}/{event_id}_SeismicStationrecords_{version}.pdf", bbox_inches='tight')

    

In [ ]:
if instruments_data_map:


    from modules.SHAKEstats import *
    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    result = plotdata_empirical_cdf(
        event_id=event_id,
        df=instruments_data,
        column='pga',unit='cm/s²',
        log_transform=True,
        #compare_dists=['lognorm', 'gamma'],
        compute_best_fit=True,
        marker_color='black',
        show_title=False,
        marker_size=10,
        output_path='./export',
        save_formats=['png', 'pdf'],
        name='empiricalcdf_pga',
        figsize=(6, 4),    
        cdf_marker='o',
        linewidth=1, y_label = ' Cumulative Probability'
    
    )
    
    
    print("Best fit distribution:", result['best_fit'])
    print("KS statistic:", result['ks_stat'])


In [ ]:
if instruments_data_map:
    
    
    from modules.SHAKEstats import *
    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    results = plotdata_residuals(
        df=instruments_data,
        x='distance',
        y='pga',
        vs30_col='vs30',
        log_y=True,
        marker='^',
        color='green',
        s=120,
        output_path='./export',save_formats=['png', 'pdf'],
        event_id=event_id,
        name='residuals_pga',unit_y='cm/s2', x_ticks=[10,50,100,200,400],
        show_title=False, 
        x_label='Epicentral Distance [$km$]',
        y_label='log(PGA) [$cm/s²$]',
        markeredge_width=1,
        figsize=(6,4),legend_loc='lower right',legend_marker_size= 10, legend_marker_edge= 2) #, legend_loc='upper left'


In [ ]:
if instruments_data_map:

    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    plotdata_qq_pp(
        df=instruments_data,
        column='pga',    log_if_skewed=True,
        dist_name='normal',  # or None to auto-detect
        name_prefix='quantile_pga',
        output_path='./export',
        event_id=event_id,
        save_formats=['png', 'pdf'],
        figsize=(6, 4), 
        show_title=False,marker_size=10
    )


In [ ]:
if instruments_data_map:
    
    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    
    result = plotdata_histogram(
        df=instruments_data,
        column='pga',
        log_transform=True,
        unit='cm/s²',
        scoring_method='ks',  # or 'aic', 'bic', or None
        output_path='./export',
        save_formats=['png', 'pdf'], 
        figsize = (6,4),
        event_id=event_id, name = 'histogramPDF_pga', 
        linewidth= 1, 
        alpha = 0.6,
        show_title = False,legend_loc='upper left')

In [ ]:
if instruments_data_map:
    
    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    ci_method_used = 'residual'
    
    results = plotdata_attenuation(
        df=instruments_data,
        vs30_col='vs30',
        fit='nonlinear',
        ci_method=ci_method_used,   # Try: 'binwise', 'loess', or 'residual'
        log_x=True,
        log_y=True,
        show_confidence=True,
        unit_y='cm/s²',
        output_path='./export',
        event_id=event_id,
        name=f'attenuation_pga_{ci_method_used}',
        return_values=True,
        s=150,
        marker='^',  # 'o' for circle, '^' for triangle, 's' for square
        figsize=(6, 4),alpha=1,
        save_formats=['png','pdf'],x_ticks=[10,50,100,200,400],show_title=False,
        legend_marker_size=10,
        markeredge_width=1, x_label = 'Epicentral Distance [$km$]')
    
    print("Params:", results['params'])
    print("Residuals:", results['residuals'][:5])

In [ ]:
# Example usage:
if instruments_data_map:
    mapper = SHAKEmapper()
    mapper.set_extent([118.52495, 124.72505, 20.94165, 26.60835]) if event_id == "us7000m9g4" else mapper.set_extent(map_extent)

    #mapper.set_extent(map_extent)
    
    fig, ax = mapper.create_basemap()  #([90.4, 103.4, 13.0, 26.8])
    
    #mapper.add_vs30_layer()
    plt.style.use('./bins/latex_shakemap_style.mplstyle')


    mapper.add_vs30_layer()
    mapper.add_stations(instruments_data['longitude'],instruments_data['latitude'])
        

    mapper.add_cities(population=1000000)
    
    mapper.update_legend(alpha=0.9)

    mapper.add_epicenter(float(shakemap_metadata['event']['lon']),float(shakemap_metadata['event']['lat']))


    plt.savefig(f"{output_path}/{event_id}_SeismicStations_map_{version}.png", bbox_inches='tight')
    #plt.savefig(f"{output_path}/{event_id}_SeismicStations_map_{version}.pdf", bbox_inches='tight')



    
    plt.show()

## Get DYFI? Data 

### DYFI? data from stationlist.json

In [ ]:
# Parse Instruments Data 
dyfi_parser = USGSParser(parser_type = 'instrumented_data',json_file=file_path_instrument_data)
dyfi_data_stationslist = dyfi_parser.get_dataframe(value_type='mmi')

print(dyfi_data_stationslist.columns)

# this will turn non‐parseable values (like "null") into actual NaN
dyfi_data_stationslist['intensity'] = pd.to_numeric(
    dyfi_data_stationslist['intensity'], errors='coerce'
)  # :contentReference[oaicite:0]{index=0}

# also ensure 'distance' is numeric
dyfi_data_stationslist['distance'] = pd.to_numeric(
    dyfi_data_stationslist['distance'], errors='coerce'
)

# drop any rows that became NaN in either column
dyfi_data_stationslist.dropna(
    subset=['intensity', 'distance'], inplace=True
)


dyfi_data_stationslist.iloc[9]

In [ ]:
dyfi_data_stationslist['

In [ ]:
from modules.SHAKEstats import *

if dyfi_stationslist_data_map:

    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    results = plotdata_residuals(
        df=dyfi_data_stationslist,
        x='distance',
        y='intensity',
        vs30_col='vs30',
        log_y=True,
        marker='o',
        color='green',
        s=150,
        output_path='./export',save_formats=['png', 'pdf'],
        event_id=event_id,
        name='residuals_dyfistationslist_mmi',unit_y='MMI', x_ticks=[10,100,200,400],
        show_title=False, 
        x_label='Epicentral Distance [$km$]',
        y_label='Residuals [$log(MMI)$]',
        legend_loc='lower right',     markeredge_width=1,
        figsize=(6, 4))

In [ ]:
if dyfi_stationslist_data_map:

    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    result = plotdata_empirical_cdf(
        event_id=event_id,
        df=dyfi_data_stationslist,
        column='intensity',unit='MMI',
        log_transform=False,
        #compare_dists=['lognorm', 'gamma'],
        compute_best_fit=True,
        marker_color='black',
        show_title=False,
        marker_size=10,
        output_path='./export',
        save_formats=['png', 'pdf'],name='empiricalcdf_dyfistationslist_mmi',
        figsize=(6, 4),    
        cdf_marker='o',
        linewidth=1, x_label = 'Intensity [MMI]', y_label = ' Cumulative Probability'
    
    )
    
    
    print("Best fit distribution:", result['best_fit'])
    print("KS statistic:", result['ks_stat'])

In [ ]:
if dyfi_stationslist_data_map:
    plt.style.use('./bins/datastats_analysis.mplstyle')

    
    plotdata_qq_pp(
        df=dyfi_data_stationslist,
        column='intensity',
        log_if_skewed=True,
        dist_name='normal',  # or None to auto-detect
        name_prefix='quantile_dyfistationslist_mmi',
        output_path='./export',
        event_id=event_id,
        save_formats=['png'],figsize=(12, 7), show_title=False
    )



In [ ]:
if dyfi_stationslist_data_map:
    plt.style.use('./bins/datastats_analysis.mplstyle')

    
    result = plotdata_histogram(
        df=dyfi_data_stationslist,
        column='intensity',
        log_transform=False,
        unit='MMI',
        scoring_method='ks',  # or 'aic', 'bic', or None
        output_path='./export',
        save_formats=['png', 'pdf'], 
        figsize = (6,4),
        event_id=event_id, name = 'HistogramPDF_dyfistationslist_mmi', 
        linewidth= 1, 
        alpha = 0.6,
        show_title = False, x_label = 'Intensity [$MMI$]'
    )


In [ ]:
if dyfi_stationslist_data_map:

    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    ci_method_used = 'residual'
    
    results = plotdata_attenuation(
        df=dyfi_data_stationslist,
        x= 'distance',
        y= 'intensity',
        vs30_col='vs30',
        fit='nonlinear',
        ci_method=ci_method_used,   # Try: 'binwise', 'loess', or 'residual'
        log_x=True,
        log_y=True,
        show_confidence=True,
        unit_y='MMI',
        output_path='./export',
        event_id=event_id,
        name=f'attenuation_dyfisttaionslist_mmi_{ci_method_used}',
        return_values=True,
        s=150,
        marker='o',  # 'o' for circle, '^' for triangle, 's' for square
        figsize=(6, 4),alpha=1,
        save_formats=['png', 'pdf'],
        x_ticks=[10,50,100,200,400],
        y_ticks=[3,4,5,6,8,10],
        show_title=False,legend_marker_size=10, x_label ='Epicentral Distance [$km$]', y_label = 'Intensity [$MMI$]',markeredge_width=1)
    
    print("Params:", results['params'])
    print("Residuals:", results['residuals'][:5])

In [ ]:
if dyfi_stationslist_data_map:

    
    import matplotlib.ticker as ticker
    
    
    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    from modules.SHAKEtools import contour_scale
    
    cmap,bounds, ticks, norm, used_scale = contour_scale('mmi')
    
    
    # this will turn non‐parseable values (like "null") into actual NaN
    dyfi_data_stationslist['intensity'] = pd.to_numeric(
        dyfi_data_stationslist['intensity'], errors='coerce'
    )  # :contentReference[oaicite:0]{index=0}
    
    # also ensure 'distance' is numeric
    dyfi_data_stationslist['distance'] = pd.to_numeric(
        dyfi_data_stationslist['distance'], errors='coerce'
    )
    
    # drop any rows that became NaN in either column
    dyfi_data_stationslist.dropna(
        subset=['intensity', 'distance'], inplace=True
    )
    
    
    
    
    # Create scatter plot
    fig, ax = plt.subplots()
    sc = ax.scatter(
        dyfi_data_stationslist['distance'], dyfi_data_stationslist['intensity'],
        c=np.asarray(dyfi_data_stationslist['intensity']), cmap=cmap, norm=norm,
        marker='o',
        s=150,edgecolors='black',
        linewidths=1, label = 'DYFI?'
    )
    
    
    
    #plt.plot(dyfi_data['dist'],dyfi_data['intensity'],
    #                              '.',mfc='gray',markersize=20, label = 'Intenisty Component')
    
    
    #set label sizes 
    plt.xlabel('Epicentral Distance [$km$]')
    plt.ylabel('MMI')
    
    
    # Add colorbar for intensity
    cb = plt.colorbar(sc, ax=ax)
    cb.set_label('MMI')
    
    
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    
    
    # Now save the figures
    plt.savefig(f"{output_path}/{event_id}_dyfi_stationslist_{version}.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"{output_path}/{event_id}_dyfi_stationslist_{version}.pdf", bbox_inches='tight', dpi=300)
    
    
    
    plt.show()

In [ ]:
# Example usage:
if dyfi_stationslist_data_map:
    mapper = SHAKEmapper()
    mapper.set_extent([118.52495, 124.72505, 20.94165, 26.60835]) if event_id == "us7000m9g4" else mapper.set_extent(map_extent)
    
    fig, ax = mapper.create_basemap()  #([90.4, 103.4, 13.0, 26.8])
    
    #mapper.add_vs30_layer()
    plt.style.use('./bins/latex_shakemap_style.mplstyle')


    mapper.add_vs30_layer()
    mapper.add_dyfi(dyfi_data_stationslist['longitude'],dyfi_data_stationslist['latitude'],dyfi_data_stationslist['intensity'])
    mapper.add_epicenter(float(shakemap_metadata['event']['lon']),float(shakemap_metadata['event']['lat']))
  

    mapper.add_cities(population=1000000)
    
    mapper.update_legend(alpha=0.9)

    plt.savefig(f"{output_path}/{event_id}_dyfi_data_stationslist_map_{version}.png", bbox_inches='tight', dpi=300)
    #plt.savefig(f"{output_path}/{event_id}_dyfi_data_stationslist_map_{version}.pdf", bbox_inches='tight', dpi=300)


    
    plt.show()


### DYFI? from CDI 

In [ ]:
# Parse did you feel it data 

dyfi_parser = USGSParser(parser_type='dyfi_data', file_path=file_path_dyfixml)
dyfi_data = dyfi_parser.get_dataframe()
dyfi_data.iloc[10]

In [ ]:
dyfi_data

In [ ]:
# Step 1: Calculate epicentral distance for each point
dyfi_data["Epicentral distance"] = haversine_distance(
    lon1=dyfi_data["Longitude"],
    lat1=dyfi_data["Latitude"],
    lon2=event_epicenter_lon,
    lat2=event_epicenter_lat)


# Step 2: Filter the DataFrame to remove points more than 400 km from the epicenter
dyfi_data_filtered = dyfi_data[dyfi_data["Epicentral distance"] <= 500].copy()


from modules.SHAKEmapper import *

# Initialize mapper
mapper = SHAKEmapper()

# Extract Vs30 and assign as a new column in dyfi_data
dyfi_data_filtered["vs30"] = mapper.extract_vs30_data(
    lons=dyfi_data_filtered["Longitude"],
    lats=dyfi_data_filtered["Latitude"]
)["point_values"]


# Normalize string-based NULL representations to real NaN
dyfi_data_filtered = dyfi_data_filtered.replace(
    ["NULL", "null", "None", "none", "", "nan", "NaN"],
    np.nan
)

# Remove rows with any remaining NaN / None in critical columns
dyfi_data_filtered = (
    dyfi_data_filtered
    .dropna(subset=[
        "Longitude",
        "Latitude",
        "Epicentral distance",
        "vs30"
    ])
    .reset_index(drop=True)
)



dyfi_data_filtered

In [ ]:
if dyfi_CDI_data_map:
    
    import numpy as np
    from matplotlib.cm import ScalarMappable
    from matplotlib.lines import Line2D
    
    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    # --- get your colormap and normalization as before ---
    cmap, bounds, ticks, norm, used_scale = contour_scale('mmi')
    
    # the data
    x    = dyfi_data['Hypocentral distance'].values
    y    = dyfi_data['CDI'].values
    resp = dyfi_data['No. of responses'].values
    
    # masks
    mask_low  = resp < 3
    mask_high = resp >= 3
    
    fig, ax = plt.subplots()
    
    # 1) plot excluded (gray) points first, at lower z-order
    ax.scatter(
        x[mask_low], y[mask_low],
        facecolor='gray', edgecolor='black',
        linewidths=1, s=120, zorder=1
    )
    
    # 2) plot included (colormapped) points on top
    sc = ax.scatter(
        x[mask_high], y[mask_high],
        c=y[mask_high], cmap=cmap, norm=norm,
        edgecolor='black', linewidths=1, s=120,
        zorder=2
    )
    
    # 3) add a colorbar for the included points
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array(y[mask_high])
    cb = plt.colorbar(sm, ax=ax)
    cb.set_label('MMI')
    
    # axes styling
    ax.set_xlabel('Epicentral Distance [$km$]')
    ax.set_ylabel('MMI')
    ax.set_xlim(0, 1000)
    
    ax.set_axisbelow(True)
    ax.grid(True, linestyle='--', color='gray', linewidth=0.3, alpha=0.7)
    
    # 4) custom legend as circles
    legend_handles = [
        Line2D([0], [0],
               marker='o', linestyle='',
               markersize=10,
               markerfacecolor='gray',
               markeredgecolor='black',
               label='Excluded (nresp < 3)'),
        Line2D([0], [0],
               marker='o', linestyle='',
               markersize=10,
               markerfacecolor='white',
               markeredgecolor='black',
               label='Included (nresp ≥ 3)')
    ]
    ax.legend(handles=legend_handles, loc='upper right')
    
    plt.tight_layout()
    plt.savefig(f"{output_path}/{event_id}_dyfi_cdi_{version}.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"{output_path}/{event_id}_dyfi_cdi_{version}.pdf", bbox_inches='tight', dpi=400)
    plt.show()


In [ ]:
if dyfi_CDI_data_map:
    
    
    from modules.SHAKEstats import *
    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    results = plotdata_residuals(
        df=dyfi_data_filtered,
        x='Epicentral distance',
        y='CDI',nresp='No. of responses',
        vs30_col='vs30',
        log_y=True,
        marker='o',
        color='green',
        s=120,
        output_path='./export',save_formats=['png', 'pdf'],
        event_id=event_id,
        name='residuals_dyficdi_mmi',unit_y='MMI', x_ticks=[10,100,200,400],
        show_title=False, 
        x_label='Epicentral Distance [$km$]',
        y_label='Residuals (DYFI? [$log(MMI)$])',
        legend_loc='lower right', 
        markeredge_width = 1,
        figsize=(6,4))

In [ ]:
if dyfi_CDI_data_map:
    
    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    result = plotdata_empirical_cdf(
        event_id=event_id,
        df=dyfi_data_filtered,
        column='CDI',unit='MMI', nresp='No. of responses',
        log_transform=False,
        #compare_dists=['lognorm', 'gamma'],
        compute_best_fit=True,
        marker_color='black',
        show_title=False,
        marker_size=10,
        output_path='./export',
        save_formats=['png', 'pdf'],name='empiricalcdf_dyficdi_mmi',
        figsize=(6,4),    
        cdf_marker='o',
        linewidth=2, x_label='DYFI? [MMI]', legend_loc = 'lower right', y_label = ' Cumulative Probability'
    
    )
    
    
    print("Best fit distribution:", result['best_fit'])
    print("KS statistic:", result['ks_stat'])


In [ ]:
if dyfi_CDI_data_map:

    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    plotdata_qq_pp(
        df=dyfi_data_filtered,
        column='CDI',
        log_if_skewed=True,
        dist_name='normal',  # or None to auto-detect
        name_prefix='quantile_dyficdi_mmi',
        output_path='./export',
        event_id=event_id,
        save_formats=['png'],figsize=(6, 4), show_title=False
    )


In [ ]:
if dyfi_CDI_data_map:

    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    
    
    result = plotdata_histogram(
        df=dyfi_data_filtered,
        column='CDI',nresp='No. of responses',
        log_transform=False,distributions= all_dists,
        unit='MMI',
        scoring_method='ks',  # or 'aic', 'bic', or None
        output_path='./export',
        save_formats=['png','pdf'], 
        figsize = (6,4),
        event_id=event_id, name = 'HistogramPDF_dyficdit_mmi', 
        linewidth= 2, 
        alpha = 0.6,
        show_title = False, legend_loc='upper right', x_label = 'Intensity [$MMI$]')

In [ ]:
if dyfi_CDI_data_map:

    
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    
    result = plotdata_histogram(
        df=dyfi_data_filtered,
        column='CDI',
        log_transform=False,distributions= all_dists,
        unit='MMI',
        scoring_method='ks',  # or 'aic', 'bic', or None
        output_path='./export',
        save_formats=['png','pdf'], 
        figsize = (6,4),
        event_id=event_id, name = 'HistogramPDF_dyficdit_alldata_mmi', 
        linewidth= 2, 
        alpha = 0.6,
        show_title = False, legend_loc='upper right', x_label = 'Intensity [$MMI$]')

In [ ]:
if dyfi_CDI_data_map:
    
    
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    
    # Apply your preferred style
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    #dyfi_data_responses_unresp_3 = dyfi_data_filtered.loc[dyfi_data_filtered['No. of responses'] >= 3, 'CDI'].copy() #dyfi_data_filtered[dyfi_data_filtered['No. of responses'] >= 3].copy()
    
    #dyfi_data_responses_lnresp_3 = dyfi_data_filtered.loc[dyfi_data_filtered['No. of responses'] < 3, 'CDI'].copy() #dyfi_data_filtered[dyfi_data_filtered['No. of responses'] < 3].copy()
    
    
    # Prepare data slices from your filtered DYFI/instrumental datasets
    data1 = dyfi_data_stationslist['intensity']
    data2 = dyfi_data_filtered.loc[dyfi_data_filtered['No. of responses'] >= 3, 'CDI'].copy()
    data3 = dyfi_data_filtered.loc[dyfi_data_filtered['No. of responses'] < 3, 'CDI'].copy()
    
    data4 = dyfi_data_filtered['CDI']
    
    datasets = [data1, data2, data3,data4]
    column_names = ['Used DYFI?', 'nresp > 3', 'nresp < 3','All DYFI?']
    
    
    # Call the compare histogram function
    fig, result = plotdata_compare_histogram(
        datasets=datasets,
        column_names=column_names,
        log_transform=False,
        unit='MMI',
        scoring_method='ks',  # or 'aic', 'bic'
        output_path='./export',
        save_formats=['png','pdf'],
        figsize=(6, 4),
        event_id=event_id,
        save_name='HistogramCompare_DYFI4',
        linewidth=2,
        alpha=0.6,
        show_title=False,
        legend_loc='upper right',
        x_label='Intensity [$MMI$]',
        y_label='Density'
    )


In [ ]:
if dyfi_CDI_data_map:
    
    
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    
    # Apply your preferred style
    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    #dyfi_data_responses_unresp_3 = dyfi_data_filtered.loc[dyfi_data_filtered['No. of responses'] >= 3, 'CDI'].copy() #dyfi_data_filtered[dyfi_data_filtered['No. of responses'] >= 3].copy()
    
    #dyfi_data_responses_lnresp_3 = dyfi_data_filtered.loc[dyfi_data_filtered['No. of responses'] < 3, 'CDI'].copy() #dyfi_data_filtered[dyfi_data_filtered['No. of responses'] < 3].copy()
    
    
    # Prepare data slices from your filtered DYFI/instrumental datasets
    data2 = dyfi_data_filtered.loc[dyfi_data_filtered['No. of responses'] >= 3, 'CDI'].copy()
    data3 = dyfi_data_filtered.loc[dyfi_data_filtered['No. of responses'] < 3, 'CDI'].copy()
    
    data4 = dyfi_data_filtered['CDI']
    
    datasets = [data2, data3,data4]
    column_names = ['nresp > 3', 'nresp < 3','DYFI?']
    
    
    
    # Call the compare histogram function
    fig, result = plotdata_compare_histogram(
        datasets=datasets,
        column_names=column_names,
        log_transform=False,
        unit='MMI',
        scoring_method='ks',  # or 'aic', 'bic'
        output_path='./export',
        save_formats=['png','pdf'],
        figsize=(6, 4),
        event_id=event_id,
        save_name='HistogramCompare_DYFI3',
        linewidth=2,
        alpha=0.6,
        show_title=False,
        legend_loc='upper right',
        x_label='Intensity [$MMI$]',
        y_label='Density'
    )


In [ ]:
if dyfi_CDI_data_map:
    
    
    results = plotdata_attenuation(
        df=dyfi_data_filtered,
        x= 'Epicentral distance',
        y= 'CDI',
        vs30_col='vs30',nresp='No. of responses',
        fit='nonlinear',
        log_x=True,
        log_y=True,
        show_confidence=True,
        unit_y='MMI',
        output_path='./export',
        event_id=event_id,
        name='attenuation_dyfidata_mmi',
        return_values=True, s=120, 
        figsize = (6,4),
        marker='o',  # 'o' for circle, '^' for triangle, 's' for square
        save_formats=['png','pdf'], y_ticks=[1,2,3,4,5,6,8,10], x_ticks = [10,100,200,400], legend_loc='best', show_title = False, 
        x_label = 'Epicentral Distance [km]', 
        y_label = 'Intensity [$MMI$]',legend_marker_size=10,markeredge_width=1
    
    )
    
    
    
    print("Params:", results['params'])
    print("Residuals:", results['residuals'][:5])



In [ ]:
%%time


# Example usage:
if dyfi_CDI_data_map:
    mapper = SHAKEmapper()
    mapper.set_extent([118.52495, 124.72505, 20.94165, 26.60835]) if event_id == "us7000m9g4" else mapper.set_extent(map_extent)
    
    fig, ax = mapper.create_basemap()  #([90.4, 103.4, 13.0, 26.8])
    
    #mapper.add_vs30_layer()
    plt.style.use('./bins/latex_shakemap_style.mplstyle')


    mapper.add_vs30_layer()
    mapper.add_dyfi(dyfi_data['Longitude'],dyfi_data['Latitude'],dyfi_data['CDI'])
        
    mapper.add_epicenter(float(shakemap_metadata['event']['lon']),float(shakemap_metadata['event']['lat']))

    mapper.add_cities(population=1000000)
    
    mapper.update_legend(alpha=0.9)

    plt.savefig(f"{output_path}/{event_id}_dyfi_data_cdi_map_{version}.png", bbox_inches='tight', dpi=300)
    #plt.savefig(f"{output_path}/{event_id}_dyfi_data_cdi_map_{version}.pdf", bbox_inches='tight', dpi=400)


    
    plt.show()

## Get Shakemap

### Shakemap First Version 

In [ ]:
shakemap_parser_v1 = USGSParser(parser_type="shakemap_xml", imt='mmi', xml_file=xml_firstshakemap_file_path)
shakemap_metadata_v1 = shakemap_parser.get_metadata()

shakemap_metadata_v1

In [ ]:
# Example usage:
if plot_shakemap_v1:
    #mapper.add_vs30_layer()
    plt.style.use('./bins/latex_shakemap_style.mplstyle')

    mapper = SHAKEmapper()

    
    fig, ax = mapper.create_basemap(figsize=(6, 6),label_size = 10)  #([90.4, 103.4, 13.0, 26.8])
    
    mapper.add_usgs_shakemap(shakemap_parser_v1,cbar_labelsize=15,cbar_ticksize=15,cbar_shrink=0.7, cbar_label = 'MMI' )

    #mapper.add_dyfi(dyfi_data['Longitude'],dyfi_data['Latitude'],dyfi_data['CDI'])

    #x_coords, y_coords = parser.get_rupture_xy()

    mapper.add_cities(population=1000000,label_fontsize=7,markersize=3)

    mapper.add_epicenter(float(shakemap_metadata['event']['lon']),float(shakemap_metadata['event']['lat']), markersize=12)
    
    mapper.update_legend()

    plt.savefig(f"{output_path}/{event_id}_mmishakemap_{first_version}.png", bbox_inches='tight', dpi=300)
    #plt.savefig(f"{output_path}/{event_id}_mmishakemap_{first_version}.pdf", bbox_inches='tight', dpi=300)


    
    plt.show()

### MMI Shakeamp Final Version

In [ ]:
shakemap_parser = USGSParser(parser_type="shakemap_xml", imt='mmi', xml_file=xml_shakemap_file_path)
shakemap_metadata = shakemap_parser.get_metadata()

shakemap_metadata

In [ ]:
# Example usage:
if plot_shakemap_stationslist_vf:
    #mapper.add_vs30_layer()
    plt.style.use('./bins/latex_shakemap_style.mplstyle')

    mapper = SHAKEmapper()

    
    fig, ax = mapper.create_basemap(figsize=(6, 6),label_size =10)  #([90.4, 103.4, 13.0, 26.8])

    mapper.add_usgs_shakemap(shakemap_parser,cbar_labelsize=15,cbar_ticksize=15,cbar_shrink=0.7, cbar_label = 'MMI' )

    # --- DYFI: clean copies of lon/lat/intensity (do NOT touch original df) ---
    null_like = {"null", "NULL", "nan", "NaN", "none", "None", "", " "}
    lon = pd.to_numeric(dyfi_data_stationslist["longitude"].replace(list(null_like), np.nan), errors="coerce")
    lat = pd.to_numeric(dyfi_data_stationslist["latitude"].replace(list(null_like), np.nan), errors="coerce")
    val = pd.to_numeric(dyfi_data_stationslist["intensity"].replace(list(null_like), np.nan), errors="coerce")

    m = lon.notna() & lat.notna() & val.notna()
    mapper.add_dyfi(lon[m].to_numpy(), lat[m].to_numpy(), val[m].to_numpy(),s=50)
    # -------------------------------------------------------------


    
    mapper.add_stations(instruments_data['longitude'],instruments_data['latitude'],s=30)

    #x_coords, y_coords = parser.get_rupture_xy()
    mapper.add_rupture(rupture_x_coords,rupture_y_coords )

    mapper.add_cities(population=1000000,label_fontsize=7,markersize=3)

    mapper.add_epicenter(float(shakemap_metadata['event']['lon']),float(shakemap_metadata['event']['lat']), markersize=12)
    
    mapper.update_legend(alpha=0.9)


    mapper.set_extent([118.52495, 124.72505, 20.94165, 26.60835]) if event_id == "us7000m9g4" else mapper.set_extent(map_extent)

    plt.savefig(f"{output_path}/{event_id}_mmishakemap_{version}.png", bbox_inches='tight', dpi=300)
    #plt.savefig(f"{output_path}/{event_id}_mmishakemap_{version}.pdf", bbox_inches='tight', dpi=300)


    
    plt.show()

### MMI Shakeamp with CDI 

In [ ]:
%%time


# Example usage:
if plot_shakemap_CDI_vf:
    #mapper.add_vs30_layer()
    plt.style.use('./bins/latex_shakemap_style.mplstyle')

    mapper = SHAKEmapper()

    fig, ax = mapper.create_basemap(figsize=(6, 6),label_size = 10)  #([90.4, 103.4, 13.0, 26.8])
    
    mapper.add_usgs_shakemap(shakemap_parser,cbar_labelsize=15,cbar_ticksize=15,cbar_shrink=0.7, cbar_label = 'MMI' )

    #mapper.add_dyfi(dyfi_data['Longitude'],dyfi_data['Latitude'],dyfi_data['CDI'])
    mapper.add_dyfi(dyfi_data['Longitude'],dyfi_data['Latitude'],dyfi_data['CDI'], nresp=dyfi_data['No. of responses'],s=50)

    #mapper.add_stations(instruments_data['longitude'],instruments_data['latitude'])

    #x_coords, y_coords = parser.get_rupture_xy()
    mapper.add_rupture(rupture_x_coords,rupture_y_coords)

    mapper.add_cities(population=1000000,label_fontsize=7,markersize=3)

    mapper.add_epicenter(float(shakemap_metadata['event']['lon']),float(shakemap_metadata['event']['lat']), markersize=12)
    
    mapper.update_legend(alpha=0.9)

    plt.savefig(f"{output_path}/{event_id}_mmishakemapCDI_{version}.png", bbox_inches='tight', dpi=300)
    #plt.savefig(f"{output_path}/{event_id}_mmishakemapCDI_{version}.pdf", bbox_inches='tight', dpi=400)

    
    plt.show()

### MMI Shakeamp with CDI 

### PGA Shakemap 

In [ ]:
shakemap_parser = USGSParser(parser_type="shakemap_xml", imt='pga', xml_file=xml_shakemap_file_path)
shakemap_metadata = shakemap_parser.get_metadata()

In [ ]:
%%time


# Example usage:
if plot_pgashakemap_vf:
    #mapper.add_vs30_layer()
    plt.style.use('./bins/latex_shakemap_style.mplstyle')

    mapper = SHAKEmapper()

    fig, ax = mapper.create_basemap(figsize=(6, 6),label_size = 10)  #([90.4, 103.4, 13.0, 26.8])

    mapper.add_usgs_shakemap(shakemap_parser,cbar_labelsize=15,cbar_ticksize=15,cbar_shrink=0.7, cbar_label = 'PGA ($cm/s^2)$' )

    #mapper.add_dyfi(dyfi_data['Longitude'],dyfi_data['Latitude'],dyfi_data['CDI'])
    mapper.add_dyfi(dyfi_data_stationslist['longitude'],dyfi_data_stationslist['latitude'],dyfi_data_stationslist['intensity']
                   ,s=50)

    mapper.add_stations(instruments_data['longitude'],instruments_data['latitude'],s=30)

    #x_coords, y_coords = parser.get_rupture_xy()
    mapper.add_rupture(rupture_x_coords,rupture_y_coords )

    mapper.add_cities(population=1000000,label_fontsize=7,markersize=3)

    mapper.add_epicenter(float(shakemap_metadata['event']['lon']),float(shakemap_metadata['event']['lat']), markersize=12)
    
    mapper.update_legend(alpha=0.9)

    mapper.set_extent([118.52495, 124.72505, 20.94165, 26.60835]) if event_id == "us7000m9g4" else mapper.set_extent(map_extent)


    plt.savefig(f"{output_path}/{event_id}_pgashakemap_{version}.png", bbox_inches='tight', dpi=300)
    #plt.savefig(f"{output_path}/{event_id}_pgaishakemap_{version}.pdf", bbox_inches='tight', dpi=400)




    
    plt.show()

In [ ]:
##import sys 
#sys.exit()

In [ ]:
mapper.get_extent()


# Shakemap Temporal Evolution


## Set Up Enviroments And Shakemap Data files

In [ ]:
from modules.SHAKEtime import *
#from pathlib import Path



## Get SHAKEtime Summary 

### Computing SHAKEsummary 

In [ ]:
if not summary_cache:
    shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
    shake.get_shake_summary(version_list)
      #add further information
    shake.add_shakemap_pgm()
    shake.add_shakemap_stdpgm()
    shake.add_rate_to_summary(version_list=version_list, metric="mmi", use_cache=False)
    shake.add_rate_to_summary(version_list=version_list, metric="pga", use_cache=False)
    shake.add_rate_to_summary(version_list=version_list, metric="pgv", use_cache=False)
    shake.add_rate_to_summary(version_list=version_list, metric="psa10", use_cache=False)
    
    shake.add_pager_exposure()
    shake.add_cities_impact(selected_cities)
    shake.add_alerts(version_list, alert_type="fatality")
    shake.add_alerts(version_list, alert_type="economic")

     # Define the output directory
    output_dir = Path(f"./export/SHAKEtime/{event_id}")
    
    # Create it (and any missing parents) if it doesn’t already exist
    output_dir.mkdir(parents=True, exist_ok=True)  # pathlib.Path.mkdir(parents=True, exist_ok=True) :contentReference[oaicite:0]{index=0}
    
    # Build the full path to your CSV file
    shake.export_summary(output_dir, file_type='csv')


#if Cache is created then import it 
elif summary_cache:
    shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
    csv_pathh = f"./export/SHAKEtime/{event_id}/SHAKEtime-Summary-{event_id}.csv"
    shake.import_summary(csv_pathh)

## Get Unified Grid

### Compute Unified Grid

In [ ]:
#shake.clear_grid_cache()
#shake.clear_summary_cache()
if not unigrid_cache:
    metrics= ['pga','pgv','mmi', 'psa10']
    for metric in metrics: 
        # Create it (and any missing parents) if it doesn’t already exist
        output_dir = Path(f"./export/SHAKEtime/{event_id}")
        output_dir.mkdir(parents=True, exist_ok=True)  # pathlib.Path.mkdir(parents=True, exist_ok=True) :contentReference[oaicite:0]{index=0}
    
        shake.get_rate_grid(version_list, metric=metric, use_cache = False)
        shake.export_unified_grid(output_dir, file_type='pickle')

#if Cache is created then import it 
elif unigrid_cache:
    metric = 'mmi'
    shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
    csv_pathh = f"./export/SHAKEtime/{event_id}/SHAKEtime_unified_grid_{event_id}_{metric}.pickle"
    shake.import_unified_grid(csv_pathh)
    
    #shake.get_unified_grid (use_cache = True)

### Unified Grid Mean and Median test 

In [ ]:

if unified_grid_test:

    
    plt.style.use('./bins/latex_font.mplstyle')

    unified_grid = shake.get_unified_grid(version_list, use_cache=True)
    list(unified_grid)
    

    df_mm, figs = shake.analyze_unified_grid_mean_median(
        version_list=version_list,
        metric="mmi",
        use_cache=True,
        x_ticks="version",          # or "version", "TaE_d"
        n_boot=800,
        ci=0.95,
        output_path="./export",
        save_formats=["png","pdf"], xrotation=90, marker_size= 7,
        line_width = 3,

        make_plots=True,
        close_figs=False,         # set True if you don’t want inline fig objects hanging around
        font_sizes={"labels":18, "ticks":18, "title":10, "legend":15},
        figsize=(6,4),
        diff_figsize=(6,4),
            show_title = False


    )
    df_mm


In [ ]:
if unified_grid_test: 
    plt.style.use('./bins/latex_font.mplstyle')

    import matplotlib as mpl
    mpl.rcParams["lines.linewidth"] = 1
    mpl.rcParams["lines.markersize"] = 7

    # Unified grid built/cached already    
    df_conc, figs_conc = shake.analyze_spatial_change_concentration(
        version_list=version_list,
        metric="mmi",
        x_ticks="version",
        output_path="./export",
        tol=1e-3,
        xrotation=90,
        font_sizes={"labels":15, "ticks":15, "title":10, "legend":10},
        figsize=(6,4),
        show_title=False,
        legend_loc ="upper right",


        top_fracs=(0.01, 0.05, 0.10),
        make_plots=True,
        close_figs=False,
    )



In [ ]:
if unified_grid_test:
    import matplotlib as mpl
    mpl.rcParams["lines.linewidth"] = 1
    mpl.rcParams["lines.markersize"] = 7

    
    df_dir, figs_dir = shake.analyze_directional_change(
        version_list=version_list,
        metric="mmi",
        x_ticks="version",
        output_path="./export",



        xrotation=90,
        font_sizes={"labels":18, "ticks":18, "title":10, "legend":15},
        figsize=(6,4),
        show_title=False,
        legend_loc ="upper right",

        tol=1e-3,
        make_plots=True,
        close_figs=False,
    )


In [ ]:


if unified_grid_test: 
    import matplotlib as mpl
    mpl.rcParams["lines.linewidth"] = 1
    mpl.rcParams["lines.markersize"] = 7

    df_q, figs_q = shake.analyze_quantile_convergence(
        version_list=version_list,
        metric="mmi",
        x_ticks="TaE_h",
        output_path="./export",
        quantiles=(0.05, 0.5, 0.95),

        xrotation=90,
        font_sizes={"labels":18, "ticks":18, "title":10, "legend":15},
        figsize=(6,4),
        show_title=False,
        legend_loc ="upper right",
        
        make_plots=True,
        close_figs=False,
    )

## Batch Export Plots 

### Plot All Rate Maps 

In [ ]:
if plot_ratemaps:
    metrics= ['mmi'] #['pga','pgv','mmi', 'psa10']
    for metric in metrics:  
        from modules.SHAKEtime import SHAKEtime
        
        shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
        plt.style.use('./bins/latex_shakemap_style.mplstyle')
        # Plot rate maps.
        rate_maps = shake.plot_ratemap_details(version_list, metric=metric,rupture_folder=rupture_folder,
                                               stations_folder=stations_folder, 
                                               output_path="./export",plot_colorbar=True, show_title=False, dpi = 300,
                                               use_cache=True, save_formats=["png"],
                                                  figsize=(6, 6),
                                                    label_size=15,
                                                    tick_size=15,
                                                    title_size=10,
                                                    scatter_size=2,
                                                    cbar_shrink=0.6,
                                                    cbar_labelsize=15,
                                                    cbar_ticksize=15,station_marker_size = 30,
                                                            dyfi_marker_size =30)


### Plot All SHAKEmaps

In [ ]:
if plot_shakemaps:
    metrics= ['mmi', 'pga'] #['pga','pgv','mmi']
    for metric in metrics:
        plt.style.use('./bins/latex_shakemap_style.mplstyle')
        # Plot ShakeMaps.
        shake.plot_shakemaps(version_list, metric=metric, rupture_folder=rupture_folder,stations_folder=stations_folder, output_path="./export" 
                             ,plot_colorbar=False, show_title=False, dpi=300, save_formats=["png"],mode = "shakemap")


### Plot SHAKEmap Uncertainty 

In [ ]:
if plot_shakemaps_std:
    plt.style.use('./bins/latex_shakemap_style.mplstyle')
    metrics= ['mmi', 'pga'] #['pga','pgv','mmi']
    for metric in metrics:
        shake.plot_std_maps(version_list, metric=metric, rupture_folder=rupture_folder,stations_folder=stations_folder
                            , output_path="./export", 
                            plot_colorbar=True, show_title=False, dpi=300,save_formats=["png"] )


### Plot Population Exposure 

In [ ]:
if plot_pager:
    plt.style.use('./bins/latex_font.mplstyle')

    shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
    csv_pathh = f"./export/SHAKEtime/{event_id}/SHAKEtime-Summary-{event_id}.csv"
    shake.import_summary(csv_pathh)

    # Plot population exposure.
    fig_pop, ax_pop = shake.plot_pop_exposure(version_list=version_list, output_path="./export",x_ticks='version',
                                              show_title=False,figsize= (6, 4),
                                              font_sizes = {"labels":15, "ticks":15}, grid=True,xrotation = 90,
                                             legend_fontsize=10,save_formats = ['png', 'pdf'] )
    #ax_pop.set_yscale("log")

    plt.show()    
    # plot Fatalities

    shake.plot_alerts(version_list, figsize= (6, 4),alert_type="fatality", output_path="./export",x_ticks='version',
                      show_title=False,font_sizes = {"labels":15, "ticks":15},legend_fontsize=10,xrotation = 90,
                      save_formats = ['png', 'pdf'])
    
    #plot Economic

    shake.plot_alerts(version_list,figsize= (6, 4), alert_type="economic", output_path="./export",x_ticks='version',
                      show_title=False,font_sizes = {"labels":15, "ticks":15},legend_fontsize=10,xrotation = 90,
                      save_formats = ['png', 'pdf'])


# Shakemap Evolution Sebsitivity Analysis

## Hazrad Temporal Displacement

In [ ]:
if plot_hazard_thd:
    
    plt.style.use('./bins/latex_fig_style.mplstyle')

    shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
    csv_pathh = f"./export/SHAKEtime/{event_id}/SHAKEtime-Summary-{event_id}.csv"
    shake.import_summary(csv_pathh)

    from pathlib import Path
    
    metrics = [
        'mmi', 'mmi_delta', 'mmi_rate',
        'pga', 'pga_delta', 'pga_rate',
        'pgv', 'pgv_delta', 'pgv_rate',
        'psa10', 'psa10_delta', 'psa10_rate',
        'stdmmi', 'stdpga', 'stdpgv', 'stdpsa10'
    ]
    
    for metric in metrics:
        try:
            shake.plot_thd(metric_type=metric, figsize=(6,4), x_ticks= 'version', output_path="./export",save_formats=["png",'pdf'],
                           font_sizes = {"labels": 18, "ticks": 18, "legend": 15, "title": 10},marker_size=10,
                           x_rotation=90,show_title=False,legend_loc='upper right') 
            print(f"✅ Completed HTD for metric '{metric}'")
        except Exception as e:
            print(f"❌ Failed HTD for metric '{metric}': {e}")

In [ ]:
%%time


from modules.SHAKEtime import *
from pathlib import Path

plt.style.use('default')

shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
csv_pathh = f"./export/SHAKEtime/{event_id}/SHAKEtime-Summary-{event_id}.csv"
shake.import_summary(csv_pathh)

dataframe = shake.get_dataframe()


#shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
csv_pathh = f"./export/SHAKEtime/{event_id}/SHAKEtime_unified_grid_{event_id}_mmi.pickle"
shake.import_unified_grid(csv_pathh)
#shake.get_unified_grid (use_cache = True)

## Cities Evolution 

In [ ]:
plt.style.use('./bins/latex_fig_style.mplstyle')


fig = shake.plot_city_and_global_progression(
    version_list=version_list,cities=selected_cities,figsize=(15,8),output_path='export',
    linewidth=3,legendloc="lower right",x_axis='TaE',y_limits=(1,10), plot_title=False)

## Compute Auxiliary Influences 

In [ ]:
%%time



if not aux_cache:
    res = shake.analyze_auxiliary_influences(
        version_list,
        rupture_folder="./event_data/SHAKEfetch/usgs-rupture-versions",
        station_folder="./event_data/SHAKEfetch/usgs-instruments_data-versions",
        dyfi_folder="./event_data/SHAKEfetch/usgs-instruments_data-versions",
        thresholds=[6.0, 7.0, 8.0,9.0],
        uncertainty_percentile=90.0,
        bootstrap_iters=200,
        radius_km=30,
        metric='mmi', use_cache=False
    )

    shake.export_auxiliary_results(res,output_dir='./export')

if aux_cache:
    res = shake.analyze_auxiliary_influences(
        version_list,
        rupture_folder="./event_data/SHAKEfetch/usgs-rupture-versions",
        station_folder="./event_data/SHAKEfetch/usgs-instruments_data-versions",
        dyfi_folder="./event_data/SHAKEfetch/usgs-instruments_data-versions",
        thresholds=[6.0, 7.0, 8.0],
        uncertainty_percentile=90.0,
        bootstrap_iters=200,
        radius_km=30,
        metric='mmi', use_cache=True
    )


## Plot Overview Panels 

In [ ]:
%%time
import matplotlib as mpl

mpl.rcParams['font.family']   = 'serif'         # or 'sans-serif', 'monospace', etc.
mpl.rcParams['font.serif']    = ['Times New Roman','Georgia']  # fallback list


if plot_overview_panels:
    plt.style.use('default')
    plt.style.use('./bins/latex_font.mplstyle')


    # 1. Tell Matplotlib to use LaTeX for all text
    #mpl.rcParams['text.usetex'] = True                                # :contentReference[oaicite:0]{index=0}
    
    # 2. Use a serif family and Computer Modern Roman (the default LaTeX font)
    #mpl.rcParams['font.family'] = 'serif'
   # mpl.rcParams['font.serif']  = ['Computer Modern Roman']          # :contentReference[oaicite:1]{index=1}


    shake.create_overview_panels(version_list,output_path='./export',rupture_folder=rupture_folder,stations_folder=stations_folder,
                                 use_cache=True,figsize=(24, 15),show_title=False,dpi=600)

## Quanfity Evolution 

In [ ]:
%%time

plt.style.use('./bins/datastats_analysis.mplstyle')


if quantify_evolution:
    #evo_df = shake.quantify_evolution(
    #    version_list=version_list,
    #    metric="mmi",
    #    thresholds=[6,7,8,9],
    #    uncertainty_percentile=90.0,
    #    bootstrap_iters=500
    #)
    
    # 8) Plot evolution of all diagnostics across versions
    evo_df = shake.plot_evolution(
        version_list,
        metric="mmi",
        thresholds=[6,7,8,9],
        uncertainty_percentile=90.0,font_sizes = {"title": 14, "labels": 20, "ticks": 20},
        bootstrap_iters=200,
        output_path="./export")

mpl.rcParams['font.family']   = 'serif'         # or 'sans-serif', 'monospace', etc.
mpl.rcParams['font.serif']    = ['Times New Roman','Georgia']  # fallback list



In [ ]:
if create_evolution_panel:
    plt.style.use('default')
    plt.style.use('./bins/latex_font.mplstyle')
    #plt.style.use('./bins/datastats_analysis.mplstyle')
    shake.create_evolution_panel(version_list, output_path='./export',figsize=(12,8), 
                                 font_sizes={'title': 15, 'labels': 15, 'ticks': 15},
                                 show_title=False,save_formats=['png', 'pdf'])#,
                                #grid_kwargs = {'linestyle': '--', 'alpha': 0.5})

## Chaos-Informed Metrics

In [ ]:
if compute_global_choas:

    plt.style.use('./bins/datastats_analysis.mplstyle')
    
    shake.plot_rolling_chaos(version_list, output_path='./export', figsize=(12,7), window_size=6, step=1,emb_dim=3, use_pca = True, 
                             show_title=False,save_formats=['png'])
    shake.plot_global_chaos(version_list=version_list, output_path='./export', figsize=(12,7),
                            show_title=False,save_formats=['png'])

    #res = shake.compute_chaos_metrics(
    #metric="mmi",
    #version_list=version_list,
    #emb_dim=6, mode="spatial",
    #use_pca=True,
    #plot=True,output_path='./export',save_formats=['csv'])

    plt.style.use('default')
    plt.style.use('./bins/latex_font.mplstyle')
    
    shake.plot_spatial_chaos(version_list=version_list,output_path='./export', figsize=(12,7), show_title = False, s=100,
                             colorbar_font=10,save_formats=['png'] )


## Quantify Data Influcene

In [ ]:
import matplotlib.pyplot as plt

if quantify_data_influence:

    plt.style.use('./bins/latex_font.mplstyle')

    figsize = (6,4)
    line_width=1
    marker_size=7


    # Ensure unified grid exists (cache)
    unified_grid = shake.get_unified_grid(version_list, use_cache=True)
    list(unified_grid)

    BIG = {"labels":17, "ticks":17, "title":10, "legend":12}

    # -------------------------
    # 1) Figure 1: Data availability (log y)
    # -------------------------
    fig1, ax1, df_data_avail = shake.plot_data_availability_timeseries(
        version_list=version_list,
        metric="mmi",
        x_ticks="version",          # or "TaE_h" / "TaE_d"
        output_path="./export",
        save_formats=["png",'pdf'],
        dpi=300,
        show_title=False,

        font_sizes=BIG,
        figsize=figsize,
        xrotation=90,
        legend_loc="best",
        grid=True,
        grid_kwargs={"linestyle":"--", "alpha":0.25},

        line_width=line_width,
        marker_size=marker_size,

        close_figs=False,
    )

    # -------------------------
    # 2) Figure 2: Hazard footprint evolution
    # -------------------------
    fig2, ax2, df_hazard = shake.plot_hazard_footprint_timeseries(
        version_list=version_list,
        metric="mmi",
        x_ticks="version",
        output_path="./export",
        save_formats=["png",'pdf'],
        dpi=300,
        show_title=False,
        yscale = 'log',

        font_sizes=BIG,
        figsize=figsize,
        xrotation=90,
        legend_loc="best",
        grid=True,
        grid_kwargs={"linestyle":"--", "alpha":0.25},

        line_width=line_width,
        marker_size=marker_size,

        close_figs=False,
    )




In [ ]:
import matplotlib.pyplot as plt

if quantify_data_influence:

    # -------------------------
    # 3) Figure 3: Update magnitude vs data increments (scatter)
    # NOTE: marker_size here is scatter "s" (area), not line marker size
    # -------------------------
    fig3, axes3, df_steps = shake.plot_update_magnitude_vs_data_increment(
        version_list=version_list,
        metric="mmi",
        response="mean_abs_delta",   # or "sum_abs_delta"
        tol=1e-3,
        output_path="./export",
        save_formats=["png",'pdf'],
        dpi=300,
        show_title=False,

        font_sizes=BIG,
        figsize=figsize,
        grid=True,
        grid_kwargs={"linestyle":"--", "alpha":0.25},

        marker_size=120,             # scatter area
        alpha=0.85,

        close_figs=False
    )

    # -------------------------
    # 4) Figure 4: Standardized influence ranking (|beta|)
    # -------------------------
    fig4, ax4, df_betas = shake.plot_standardized_data_influence(
        version_list=version_list,
        metric="mmi",
        tol=1e-3,
        response_set=("mean_abs_delta", "sum_abs_delta"),
        output_path="./export",
        save_formats=["png",'pdf'],
        dpi=300,
        show_title=False,

        font_sizes=BIG,
        figsize=figsize,
        grid=True,
        grid_kwargs={"linestyle":"--", "alpha":0.25},

        close_figs=False,
    )

    # -------------------------
    # 5) Figure 5: Lag effect (correlation vs lag)
    # -------------------------
    fig5, ax5, df_lag = shake.plot_data_effect_lag(
        version_list=version_list,
        metric="mmi",
        response="mean_abs_delta",
        max_lag=4,
        tol=1e-3,
        output_path="./export",
        save_formats=["png",'pdf'],
        dpi=300,
        show_title=False,

        font_sizes=BIG,
        figsize=figsize,
        legend_loc="upper right",
        grid=True,
        grid_kwargs={"linestyle":"--", "alpha":0.25},

        line_width=line_width,
        marker_size=marker_size,

        close_figs=False,
    )

    # Optional: inspect returned tables
    df_data_avail.head(), df_hazard.head(), df_steps, df_betas, df_lag


In [ ]:
import sys 
sys.exit()